In [1]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [2]:
import sys

linewidth = 100
formatter = {
    'float': lambda num: f'{num:5.1f}',
    'int': lambda num: f'{float(num):5.1f}',
}
print_args = {
#     'threshold': sys.maxsize,
    'threshold': 100,
    'linewidth': 100,
    'formatter': formatter,
    'edgeitems': 6,
}


In [3]:
import numpy as np
from qtmpy.constants import RYDBERG

alat = 5.107 * 12
latvec_alat = 0.5 * np.array([
    [ 1,  1,  1],
    [-1,  1,  1],
    [-1, -1,  1]
])

ecutwfn = 40 * RYDBERG

In [4]:
from qtmpy.lattice import RealLattice, ReciLattice

reallat = RealLattice.from_alat(alat, *latvec_alat)
recilat = ReciLattice.from_reallat(reallat)

print(reallat.axes_alat)
print(recilat.tpiba)
print(recilat.axes_tpiba)

([0.5, 0.5, 0.5], [-0.5, 0.5, 0.5], [-0.5, -0.5, 0.5])
0.10252570503197549
([1.0, 0.0, 1.0], [-1.0, 1.0, 0.0], [0.0, -1.0, 1.0])


In [5]:
from qtmpy.gspace.gspc import GSpace

gspc_rho = GSpace(recilat, 4 * ecutwfn)
print(gspc_rho.grid_shape)
print(gspc_rho.size_g)
with np.printoptions(**print_args):
    for i in range(3):
        print(f'G_{i+1}: ', gspc_rho.g_cryst[i])
    print(f'G^2: ', gspc_rho.g_norm2)

(216, 216, 216)
3932843
G_1:  [  0.0   0.0   0.0   0.0   0.0   0.0 ...  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0]
G_2:  [  0.0   0.0   0.0   0.0   0.0   0.0 ...  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0]
G_3:  [  0.0   1.0   2.0   3.0   4.0   5.0 ...  -6.0  -5.0  -4.0  -3.0  -2.0  -1.0]
G^2:  [  0.0   0.0   0.1   0.2   0.3   0.5 ...   0.8   0.5   0.4   0.2   0.1   0.0]


In [6]:
gspc_wfn = GSpace(recilat, ecutwfn)
print(gspc_wfn.size_g)
with np.printoptions(**print_args):
    for i in range(3):
        print(f'G_{i+1}: ', gspc_wfn.g_cryst[i])
    print(f'G^2: ', gspc_wfn.g_norm2)

491177
G_1:  [  0.0   0.0   0.0   0.0   0.0   0.0 ...  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0]
G_2:  [  0.0   0.0   0.0   0.0   0.0   0.0 ...  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0]
G_3:  [  0.0   1.0   2.0   3.0   4.0   5.0 ...  -6.0  -5.0  -4.0  -3.0  -2.0  -1.0]
G^2:  [  0.0   0.0   0.1   0.2   0.3   0.5 ...   0.8   0.5   0.4   0.2   0.1   0.0]


In [7]:
gspc = gspc_rho

arr_g = gspc.create_buffer_g(1)
arr_r = gspc.create_buffer_r(1)
print(arr_g.shape, arr_r.shape)

arr_g[:] = np.random.rand(gspc.size_g) \
    + 1j * np.random.rand(gspc.size_g)

arr_r[:] = 0
arr_r.put(gspc.idxgrid, arr_g)

arr_r = np.fft.ifftn(arr_r.reshape(gspc.grid_shape))
arr_r = arr_r.reshape((1, -1))

out = gspc.create_buffer_r(1)
gspc.g2r(arr_g, out)
print(np.allclose(out, arr_r))

out = gspc.create_buffer_g(1)
gspc.r2g(arr_r, out)
print(np.allclose(out, arr_g))

(1, 3932843) (1, 10077696)
False
True


In [8]:
from qtmpy.gspace.fft.full import FFT3DFull
from qtmpy.gspace.fft.sticks import FFT3DSticks
from qtmpy.gspace.fft.sticks_test import FFT3DSticksv1, FFT3DSticksv2
from qtmpy import qtmconfig
qtmconfig.fft_backend = 'pyfftw'


timeit = True
for FFT3D_ in [FFT3DFull, FFT3DSticks, FFT3DSticksv1, FFT3DSticksv2, FFT3DFull]:
    class GSpaceTest(GSpace):
        FFT3D = FFT3D_
        stick_idim = 1
    
    gspc = GSpaceTest(recilat, ecutwfn)
    arr_g = gspc.create_buffer_g(1)
    arr_r = gspc.create_buffer_r(1)

    arr_g[:] = np.random.rand(gspc.size_g) \
        + 1j * np.random.rand(gspc.size_g)

    arr_r[:] = 0
    arr_r.put(gspc.idxgrid, arr_g)
    arr_r = arr_r.reshape((-1, *gspc.grid_shape))
    arr_r[:] = np.fft.ifftn(arr_r, axes=(-1, -2, -3))
    arr_r = arr_r.reshape(-1, gspc.size_r)
    
    print(type(gspc._fft), gspc._fft.FFTBackend)
    print(getattr(gspc._fft, 'sticks_idim', None))
    out = gspc.create_buffer_r(1)
    gspc.g2r(arr_g, out)
    print(np.allclose(out, arr_r))
    
    out = gspc.create_buffer_g(1)
    gspc.r2g(arr_r, out)
    print(np.allclose(out, arr_g))
    
    if timeit:
        out = gspc.create_buffer_r(1)
        %timeit gspc.g2r(arr_g, out)
        out = gspc.create_buffer_g(1)
        %timeit gspc.r2g(arr_r, out)
    print()

<class 'qtmpy.gspace.fft.full.FFT3DFull'> <class 'qtmpy.gspace.fft.backend.pyfftw_.PyFFTWFFTWrapper'>
None
False
True
32.5 ms ± 786 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
35 ms ± 3.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

<class 'qtmpy.gspace.fft.sticks.FFT3DSticks'> <class 'qtmpy.gspace.fft.backend.pyfftw_.PyFFTWFFTWrapper'>
None
False
True
45.3 ms ± 3.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
33.5 ms ± 2.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

<class 'qtmpy.gspace.fft.sticks_test.FFT3DSticksv1'> <class 'qtmpy.gspace.fft.backend.pyfftw_.PyFFTWFFTWrapper'>
0
False
True
35.6 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
35.3 ms ± 1.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

<class 'qtmpy.gspace.fft.sticks_test.FFT3DSticksv2'> <class 'qtmpy.gspace.fft.backend.pyfftw_.PyFFTWFFTWrapper'>
0
False
True
35 ms ± 872 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
34 ms ± 1.19 m

In [9]:
from qtmpy import qtmconfig

qtmconfig.fft_backend = 'pyfftw'

gspc = GSpace(recilat, ecutwfn)
print(gspc.size_g, type(gspc._fft), gspc._fft.FFTBackend)

arr_g = gspc.create_buffer_g(1)
arr_r = gspc.create_buffer_r(1)

arr_g[:] = np.random.rand(gspc.size_g) \
    + 1j * np.random.rand(gspc.size_g)

arr_r[:] = 0
arr_r.put(gspc.idxgrid, arr_g)
arr_r = arr_r.reshape((-1, *gspc.grid_shape))
arr_r[:] = np.fft.ifftn(arr_r, axes=(-1, -2, -3))
arr_r = arr_r.reshape(-1, gspc.size_r)


491177 <class 'qtmpy.gspace.fft.full.FFT3DFull'> <class 'qtmpy.gspace.fft.backend.pyfftw_.PyFFTWFFTWrapper'>


In [10]:
profile = 'g2r'

In [11]:
from qtmpy.gspace.fft.full import FFT3DFull

class GSpaceTest(GSpace):
    FFT3D = FFT3DFull
gspc = GSpaceTest(recilat, ecutwfn)
print(type(gspc._fft), gspc._fft.FFTBackend)

if profile == 'g2r':
    out = gspc.create_buffer_r(1)
    gspc.g2r(arr_g, out)
    print(np.allclose(out, arr_r))
    %lprun -f gspc._fft.g2r gspc.g2r(arr_g, out)
else:    
    out = gspc.create_buffer_g(1)
    gspc.r2g(arr_r, out)
    print(np.allclose(out, arr_g))
    %lprun -f gspc._fft.r2g gspc.r2g(arr_r, out)

<class 'qtmpy.gspace.fft.full.FFT3DFull'> <class 'qtmpy.gspace.fft.backend.pyfftw_.PyFFTWFFTWrapper'>
False


Timer unit: 1e-09 s

Total time: 0.0333332 s
File: /home/lightpeak008/qtmpy/src/qtmpy/gspace/fft/full.py
Function: g2r at line 24

Line #      Hits         Time  Per Hit   % Time  Line Contents
    24                                               def g2r(self, arr_inp: NDArray, arr_out: NDArray) -> None:
    25         1    1736174.0 1736174.0      5.2          self._work[:] = 0
    26         1    4155791.0 4155791.0     12.5          self._work.reshape(-1)[self.idxgrid] = arr_inp
    27         1   25410205.0 25410205.0     76.2          self.worker.ifft(self.normalise_idft)
    28         1    2031022.0 2031022.0      6.1          arr_out[:] = self._work

In [12]:
from qtmpy.gspace.fft.sticks import FFT3DSticks

class GSpaceTest(GSpace):
    FFT3D = FFT3DSticks
gspc = GSpaceTest(recilat, ecutwfn)
print(type(gspc._fft), gspc._fft.FFTBackend)

if profile == 'g2r':
    out = gspc.create_buffer_r(1)
    gspc.g2r(arr_g, out)
    print(np.allclose(out, arr_r))
    %lprun -f gspc._fft.g2r gspc.g2r(arr_g, out)
else:    
    out = gspc.create_buffer_g(1)
    gspc.r2g(arr_r, out)
    print(np.allclose(out, arr_g))
    %lprun -f gspc._fft.r2g gspc.r2g(arr_r, out)

<class 'qtmpy.gspace.fft.sticks.FFT3DSticks'> <class 'qtmpy.gspace.fft.backend.pyfftw_.PyFFTWFFTWrapper'>
False


Timer unit: 1e-09 s

Total time: 0.0386248 s
File: /home/lightpeak008/qtmpy/src/qtmpy/gspace/fft/sticks.py
Function: g2r at line 48

Line #      Hits         Time  Per Hit   % Time  Line Contents
    48                                               def g2r(self, arr_inp: NDArray, arr_out: NDArray) -> None:
    49         1    1248395.0 1248395.0      3.2          self._work_sticks.fill(0)
    50         1    4157000.0 4157000.0     10.8          self._work_sticks.reshape(-1)[self.g2sticks] = arr_inp
    51         1    7592700.0 7592700.0     19.7          self.fft3.ifft(self.normalise_idft)
    52                                           
    53         1    1762026.0 1762026.0      4.6          self._work_full.fill(0)
    54         1    6907604.0 6907604.0     17.9          self._work_full.reshape((self.shape[0], -1))[
    55         1       3522.0   3522.0      0.0              (slice(None), self.sticks2full)
    56         1       1231.0   1231.0      0.0          ] = self._work_

In [13]:
class GSpaceTest(GSpace):
    FFT3D = FFT3DSticks
gspc = GSpaceTest(recilat, ecutwfn)
print(type(gspc._fft), gspc._fft.FFTBackend)

out = gspc.create_buffer_g(1)
gspc.r2g(arr_r, out)
print(np.allclose(out, arr_g))
%lprun -f gspc._fft.r2g gspc.r2g(arr_r, out)

<class 'qtmpy.gspace.fft.sticks.FFT3DSticks'> <class 'qtmpy.gspace.fft.backend.pyfftw_.PyFFTWFFTWrapper'>
True


Timer unit: 1e-09 s

Total time: 0.0302251 s
File: /home/lightpeak008/qtmpy/src/qtmpy/gspace/fft/sticks.py
Function: r2g at line 38

Line #      Hits         Time  Per Hit   % Time  Line Contents
    38                                               def r2g(self, arr_inp: NDArray, arr_out: NDArray) -> None:
    39         1    2059316.0 2059316.0      6.8          self._work_full[:] = arr_inp
    40         1   15190340.0 15190340.0     50.3          self.fft12.fft()
    41                                           
    42         1      11115.0  11115.0      0.0          work_full = self._work_full.reshape((self.shape[0], -1))
    43         1    3011220.0 3011220.0     10.0          work_full.take(self.sticks2full, axis=1, out=self._work_sticks, mode='clip')
    44         1    6792292.0 6792292.0     22.5          self.fft3.fft()
    45                                           
    46         1    3160863.0 3160863.0     10.5          self._work_sticks.take(self.g2sticks, out=arr_ou

In [14]:
from qtmpy.gspace.fft.sticks import FFT3DSticksv2

class GSpaceTest(GSpace):
    FFT3D = FFT3DSticksv2
gspc = GSpaceTest(recilat, ecutwfn)
print(type(gspc._fft), gspc._fft.FFTBackend)

if profile == 'g2r':
    out = gspc.create_buffer_r(1)
    gspc.g2r(arr_g, out)
    print(np.allclose(out, arr_r))
    %lprun -f gspc._fft.g2r gspc.g2r(arr_g, out)
else:    
    out = gspc.create_buffer_g(1)
    gspc.r2g(arr_r, out)
    print(np.allclose(out, arr_g))
    %lprun -f gspc._fft.r2g gspc.r2g(arr_r, out)

ImportError: cannot import name 'FFT3DSticksv2' from 'qtmpy.gspace.fft.sticks' (/home/lightpeak008/qtmpy/src/qtmpy/gspace/fft/sticks.py)

In [ ]:
from qtmpy.gspace.fft.sticks import FFT3DSticksv1, FFT3DSticksv1

class GSpaceTest(GSpace):
    FFT3D = FFT3DSticksv1
    
gspc = GSpaceTest(recilat, ecutwfn)
print(gspc.size, type(gspc._fft), gspc._fft.FFTBackend)

out = gspc.create_buffer_r(1)
%lprun -f gspc._fft.g2r gspc.g2r(arr_g, out)
print(np.allclose(out, arr_r))

# out = gspc.create_buffer_g(1)
# %lprun -f gspc._fft.r2g gspc.r2g(arr_r, out)
# print(np.allclose(out, arr_g))

In [ ]:
a = np.arange(24).reshape(2,3,4)

b = np.transpose(a, (0, 2, 1))
print(a.shape, b.shape)
print(a.data, b.data)
print(a.strides, b.strides)
print(b.base)
print(a.base)
print(np.shares_memory(a, b))